## Data Modeling
***

Notes
1. Dummy
2. Logreg

XGBClassifier, RandomForest

3. XGBClassifier, f2
3. XGBClassifier, f2-weighted
3. XGBClassifier, f2-SMOTE
3. XGBClassifier, f2-ADASYN

Randomize winner?

Gridsearch from there?

3. XGBClassifier, f2 - randomizedcv
3. XGBClassifier, f2-weighted - randomizedcv
3. XGBClassifier, f2-SMOTE - randomizedcv
3. XGBClassifier, f2, randomizedcv
3. XGBClassifier, f2-weighted, randomizedcv
3. XGBClassifier, f2, randomizedcv

In [27]:
import matplotlib.pyplot as plt
import pandas as pd

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score, make_scorer, fbeta_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

df = pd.read_csv('data/Cardiovascular_Diseases_Risk_Prediction_Dataset.csv')
ordinal = ['General_Health', 'Checkup', 'Age_Category']
numeric = list(df.select_dtypes(exclude=object).columns)
categorical = list(df.select_dtypes(object).columns)
for i in ordinal:
    categorical.remove(i)
categorical.remove('Heart_Disease')

# Splitting the training from the validation data
# Making sure the split is stratefied given teh imbalance of our target variable
y = df['Heart_Disease']
X = df.drop('Heart_Disease',axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=12, stratify= y)

# Transforming the target variable into 1's and 0's
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


# Listing categories in order for each ordinal variable.
health = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
check = [
    'Never', '5 or more years ago', 'Within the past 5 years',
    'Within the past 2 years', 'Within the past year'
]
age = [
    '18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80+'
]

# Instantiating an OrdinalEncoder transformer to encode ordinal variables. 
oe = OrdinalEncoder(categories=[health, check, age])


# Instantiating a OneHotEncoder transformer to be used on the categorical varaibles. 
ohe = OneHotEncoder()

# Creating a column transformer to be used in a pipeline
col_transformer = ColumnTransformer(transformers=[
    ('oe', OrdinalEncoder(categories=[health, check, age]), ordinal),
    ('ohe', OneHotEncoder(), categorical)
], remainder="passthrough")

def model_scores(model_name, model, X_train, y_train, model_list= [], notes = ''):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)
    scoring = {'f2': make_scorer(fbeta_score, beta=2),
               'recall': 'recall',
               'accuracy':'accuracy',
               'f1':'f1',
               'roc_auc':'roc_auc',
               'precision': 'precision'}
    scores = cross_validate(model,
                           X_train,
                           y_train,
                           scoring = scoring,
                           cv=skf,
                           n_jobs=-1)
    time = round(scores['fit_time'].mean() + scores['score_time'].mean(),0)
    f2 = round(scores['test_f2'].mean(),4)*100
    recall = round(scores['test_recall'].mean(),4)*100
    accuracy = round(scores['test_accuracy'].mean(),4)*100
    f1 = round(scores['test_f1'].mean(),4)*100
    roc_auc = round(scores['test_roc_auc'].mean(),4)*100
    precision = round(scores['test_precision'].mean(),4)*100
    model_list.append([model_name, time, f2, recall, accuracy, f1, roc_auc, precision, notes])
    df = pd.DataFrame(model_list, columns=['model', 'time (in s)', 'f2', 'recall', 'accuracy', 'f1', 'roc_auc', 'precision', 'notes'])
    return model_list, df


### Dummy Model

In [12]:
pipe = Pipeline(steps=[('ct', col_transformer),
                      ('dm', DummyClassifier(strategy='most_frequent'))])
dummy_model = pipe.fit(X_train, y_train)
ml, df1 = model_scores('DummyClassifier', dummy_model, X_train, y_train, model_list= [], notes= "Dummy Model")
df1

,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,DummyClassifier,1.0,0.0,0.0,91.92,0.0,50.0,0.0,Dummy Model


### Baseline Model

In [13]:
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                      ('log_reg', LogisticRegression(solver='lbfgs', max_iter=1000, random_state=12))])
logreg_model = pipe.fit(X_train, y_train)
ml, df1 = model_scores('LogisticRegression', logreg_model, X_train, y_train, model_list= ml, notes= "Baseline Model")
df1

### Model Iteration


In [24]:
pipe_rfc = Pipeline(steps=[('ct', col_transformer),
                       ('rfc', RandomForestClassifier(random_state=12))])
rfc_model = pipe_rfc.fit(X_train, y_train)
compare_list, compare_df = model_scores('RandomForestClassifier', rfc_model, X_train, y_train, model_list= [] notes= "Basic")

pipe_xgb = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_model = pipe_xgb.fit(X_train, y_train)
compare_list, compare_df = model_scores('XGBClassifier', xgb_model, X_train, y_train, model_list= compare_list notes= "Basic")
compare_df

,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,RandomForestClassifier,40.0,5.34,4.38,91.81,7.96,80.85,43.76,Basic
1,XGBClassifier,58.0,6.22,5.10,91.89,9.23,82.99,48.57,Basic


In [28]:
# Choose between XGBClassifier Variants

# Basic
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_model, X_train, y_train, model_list = [], notes= "Basic")

# Weighted
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12,scale_pos_weight=283883/24971))])
xgb_weighted = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_weighted, X_train, y_train, model_list = ml_xgb, notes= "Weighted")

# SMOTE
pipe = ImPipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                       ('sm',SMOTE(random_state=12)),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_smote = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_smote, X_train, y_train, model_list = ml_xgb, notes= "SMOTE")

# ADASYN
pipe = ImPipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                       ('sm',ADASYN(random_state=12)),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_adasyn = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores('XGBClassifier', xgb_adasyn, X_train, y_train, model_list = ml_xgb, notes= "ADASYN")
df_xgb


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,53.0,49.27,75.49,74.52,32.39,82.41,20.62,Weighted
1,XGBClassifier,144.0,6.75,5.57,91.81,9.90,82.81,44.54,SMOTE
2,XGBClassifier,336.0,6.34,5.22,91.83,9.36,82.86,45.48,ADASYN


In [29]:
pipe_xgb = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12, tree_method='hist'))])
xgb_model = pipe_xgb.fit(X_train, y_train)
blah, duh = model_scores('XGBClassifier', xgb_model, X_train, y_train, model_list= [], notes= "Basic")
duh

,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,7.0,6.38,5.24,91.86,9.43,82.94,46.74,Basic


In [ ]:
# Randomize the Weighted
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xbg', XGBClassifier(random_state=12, tree_method='hist', scale_pos_weight= 283883/24971))])

params = {
    'xbg__n_estimators': range(50,1000,50),
    'xbg__max_depth': range(1,15),
    'xbg__eta': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    'xbg__colsample_bytree': np.linspace(0,1,50),
    'xbg__min_child_weight': range(1,10),
    'xbg__gamma': [0, 0.1, 1, 10, 100, 1000],
    'xbg__reg_alpha': [0, 0.01, 0.1, 1, 10],
    'xbg__reg_lambda':[0, 0.01, 0.1, 1, 10]
}
ftwo_scorer = make_scorer(fbeta_score, beta=2)

rs = RandomizedSearchCV(estimator= pipe, param_distributions= params,
                       n_iter=300, scoring=ftwo_scorer, n_jobs=-1, cv=3, random_state=12)
rand_search = rs.fit(X_train, y_train)
rand_model = rand_search.best_estimator_
ml, df1 = model_scores('XGBClassifier', rand_model, X_train, y_train, model_list= [], notes = 'Best Estimator from RS')
print(rand_search.best_params_)
df1


In [ ]:
# GridSearching the winnfer for the final
xgb = XGBClassifier(reg_lambda= 1, 
              reg_alpha= 1,
              min_child_weight= 7,
              gamma= 0.1,
              colsample_bytree= 0.26530612244897955,
              tree_method= "hist",
              scale_pos_weight=283883/24971,
              random_state=12)

pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xbg', xgb)])

params = {'xbg__n_estimators': np.linspace(200,300,5,dtype=int),
          'xbg__max_depth': [1, 2, 3, 4, 5],
          'xbg__eta': np.linspace(0.1,0.2,5)
         }

gs = GridSearchCV(pipe, param_grid= params, scoring=ftwo_scorer, n_jobs=-1, cv=5)

g_search = gs.fit(X_train, y_train)
gs_model = g_search.best_estimator_
ml, df1 = model_scores('XGBClassifier', gs_model, X_train, y_train, model_list= [], notes = 'Best Estimator from GS')
print(g_search.best_params_)
df1


In [14]:
# 1
xgb = XGBClassifier(reg_lambda= 1, 
              reg_alpha= 1,
              n_estimators= 250,
              min_child_weight= 7,
              max_depth= 3,
              gamma= 0.1,
              eta= 0.15,
              colsample_bytree= 0.26530612244897955,
              tree_method= "hist",
              scale_pos_weight=283883/24971,
              random_state=12)

pipe = Pipeline(steps=[('ct', col_transformer),
                      ('xgb', xgb)])
xgb_model = pipe.fit(X_train, y_train)
ml, df1 = model_scores('XGBClassifier', xgb_model, X_train, y_train, model_list= ml, notes= "f2(weighted)")



### Final Model
I decided to have my final model be the step where `greenbelt` was dropped (index = 8) because the the Adjusted R-Squared was high, the Mean Absolute Error was relatively low, and the Conditional Number didn't seem to improve much as the iteration continued. 

**Target Variable:** price <br>
**Predictor Variables:** sqft_living_norm, waterfront, and 65 different zip codes.

In [ ]:
# This function does the same thing as baseline, but for the final model.
final_results, final_df = dm.final_res(df)
final_df

# I still wonder if 68 features is too many, perhaps this is why the condition 
# number is still so large. 